In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import datatable as dt
import matplotlib.pyplot as plt
import gc
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
import pickle


from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


from sklearn import set_config
set_config(display='diagram') 

from tqdm.notebook import tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.core.common.SettingWithCopyWarning)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

class PurgedGroupTimeSeriesSplitStacking(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    stacking_mode : bool, default=True
        Whether to provide an additional set to test a stacking classifier or not. 
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    max_val_group_size : int, default=Inf
        Maximum group size for a single validation set.
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split, if stacking_mode = True and None 
        it defaults to max_val_group_size.
    val_group_gap : int, default=None
        Gap between train and validation
    test_group_gap : int, default=None
        Gap between validation and test, if stacking_mode = True and None 
        it defaults to val_group_gap.
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 stacking_mode=True,
                 max_train_group_size=np.inf,
                 max_val_group_size=np.inf,
                 max_test_group_size=np.inf,
                 val_group_gap=None,
                 test_group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.max_val_group_size = max_val_group_size
        self.max_test_group_size = max_test_group_size
        self.val_group_gap = val_group_gap
        self.test_group_gap = test_group_gap
        self.verbose = verbose
        self.stacking_mode = stacking_mode
        
    def split(self, X, y=None, groups=None):
        if self.stacking_mode:
            return self.split_ensemble(X, y, groups)
        else:
            return self.split_standard(X, y, groups)
        
    def split_standard(self, X, y=None, groups=None):
        """Generate indices to split data into training and validation set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/validation set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        val : ndarray
            The validation set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_splits = self.n_splits
        group_gap = self.val_group_gap
        max_val_group_size = self.max_val_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_val_size = min(n_groups // n_folds, max_val_group_size)
        group_val_starts = range(n_groups - n_splits * group_val_size,
                                  n_groups, group_val_size)
        for group_val_start in group_val_starts:
            train_array = []
            val_array = []

            group_st = max(0, group_val_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_val_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for val_group_idx in unique_groups[group_val_start:
                                                group_val_start +
                                                group_val_size]:
                val_array_tmp = group_dict[val_group_idx]
                val_array = np.sort(np.unique(
                                              np.concatenate((val_array,
                                                              val_array_tmp)),
                                     axis=None), axis=None)

            val_array  = val_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in val_array]
            
    def split_ensemble(self, X, y=None, groups=None):
        """Generate indices to split data into training, validation and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        val : ndarray
            The validation set indices for that split (testing indices for base classifiers).
        test : ndarray
            The testing set indices for that split (testing indices for final classifier)
        """

        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
            
        X, y, groups = indexable(X, y, groups)
        n_splits = self.n_splits
        val_group_gap = self.val_group_gap
        test_group_gap = self.test_group_gap
        if test_group_gap is None:
            test_group_gap = val_group_gap
        max_train_group_size = self.max_train_group_size
        max_val_group_size = self.max_val_group_size
        max_test_group_size = self.max_test_group_size
        if max_test_group_size is None:
            max_test_group_size = max_val_group_size
            
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)

        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_val_size = min(n_groups // n_folds, max_val_group_size)
        group_test_size = min(n_groups // n_folds, max_test_group_size)
        
        group_test_starts = range(n_groups - n_splits * group_test_size, n_groups, group_test_size)
        train_indices= []
        val_indices= []
        test_indices= []
        
        for group_test_start in group_test_starts:

            train_array = []
            val_array = []
            test_array = []
            
            val_group_st = max(max_train_group_size + val_group_gap, 
                               group_test_start - test_group_gap - max_val_group_size)

            train_group_st = max(0, val_group_st - val_group_gap - max_train_group_size)

            for train_group_idx in unique_groups[train_group_st:(val_group_st - val_group_gap)]:

                train_array_tmp = group_dict[train_group_idx]

                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size

            for val_group_idx in unique_groups[val_group_st:(group_test_start - test_group_gap)]:
                val_array_tmp = group_dict[val_group_idx]
                val_array = np.sort(np.unique(
                                              np.concatenate((val_array,
                                                              val_array_tmp)),
                                     axis=None), axis=None)

            val_array  = val_array[val_group_gap:]

            for test_group_idx in unique_groups[group_test_start:(group_test_start + group_test_size)]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[test_group_gap:]

            yield [int(i) for i in train_array], [int(i) for i in val_array], [int(i) for i in test_array]

### Pickle save and load fitted model

In [ ]:
DATA_PATH = '../input/jane-street-market-prediction/'
save_path = './'
# LOAD_PATH = '../input/mlp012003weights'
# f"{CACHE_PATH}/online_model{_fold}.pth"

def save_pickle(dic, save_path):
    with open(save_path, 'wb') as f:
    # with gzip.open(save_path, 'wb') as f:
        pickle.dump(dic, f)

def load_pickle(load_path):
    with open(load_path, 'rb') as f:
    # with gzip.open(load_path, 'rb') as f:
        message_dict = pickle.load(f)
    return message_dict

In [ ]:
train = (
    dt.fread('../input/jane-street-market-prediction/train.csv')
      .to_pandas()
)
pd.set_option('display.max_columns', None)
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns})[train['date']>85]
train_85 = train[train['date']<=85]

In [ ]:
#ignoring rows with weight=0 (which are included for completeness)
train = train.query('weight > 0').reset_index(drop = True)

#Last value imputation using ffillna
features = [c for c in train.columns if 'feature' in c]
train[features] = train[features].fillna(method = 'ffill').fillna(0)
train['action'] = (train['resp'] > 0).astype('int')

#feature drop
to_be_dropped = ['feature_21','feature_24','feature_25','feature_55',
                  'feature_58','feature_121','feature_127','feature_61',
                  'feature_63','feature_5','feature_3','feature_38',
                  'feature_66','feature_69', 'feature_12', 'feature_26', 'feature_68',
                  'feature_7','feature_8','feature_17','feature_18',
                  'feature_27','feature_28','feature_72','feature_78',
                  'feature_84','feature_90','feature_96','feature_102',
                  'feature_108','feature_114',
                  'feature_35','feature_36','feature_32','feature_40',
                  'feature_48','feature_122','feature_128','feature_76',
                  'feature_110','feature_101','feature_113','feature_116',
                  'feature_107','feature_119','feature_129','feature_126'] 
#to_be_selected = [x for x in features if (x not in to_be_dropped)]
to_be_selected = ['feature_41','feature_42','feature_43', 'feature_55', 'feature_56',
                  'feature_57','feature_58','feature_59','feature_60',
                  'feature_1','feature_2','feature_3', 'feature_4','feature_5',
                  'feature_6','feature_7','feature_8','feature_9',
                  'feature_44','feature_45','feature_90','feature_92','feature_93',
                  'feature_99','feature_101','feature_119','feature_104','feature_107',
                  'feature_102','feature_105','feature_73','feature_74','feature_75',
                  'feature_120','feature_121','feature_122','feature_123','feature_124',
                  'feature_125','feature_126','feature_127','feature_128']

#train = train.drop(columns=to_be_dropped)
to_be_selected.append('date')
to_be_selected.append('action')
train = train.loc[:,to_be_selected]
f_mean = np.mean(train.loc[:,to_be_selected].values,axis=0)

In [ ]:
# X,y
X_utility = train.copy()
X = train.loc[:, [c for c in train.columns if 'feature' in c]]
#standard scaling
X = StandardScaler().fit_transform(X)
y = train.loc[:,'action']
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X), y, test_size=0.2, random_state = 42)
groups=train.iloc[X_train.index]['date'].values

In [ ]:
from matplotlib.colors import ListedColormap

name_dict = {True: 'With_Stacking_Set', 
             False: 'No_Stacking_Set'}

def plot_cv_indices_stacking(cv, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""
    
    cmap_cv = plt.cm.coolwarm

    jet = plt.cm.get_cmap('jet', 256)
    seq = np.linspace(0, 1, 256)
    _ = np.random.shuffle(seq)   # inplace
    cmap_data = ListedColormap(jet(seq))

    # Generate the training/testing visualizations for each CV split
    for ii, indices_split in enumerate(cv.split(X=X, y=y, groups=groups)):
        # Fill in indices with the training/test groups
        
        indices = np.array([np.nan] * len(X))
        indices[indices_split[0]] = 1
        indices[indices_split[1]] = 0
        if cv.stacking_mode:
            indices[indices_split[2]] = -1

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)

    # Plot the data classes and groups at the end
    ax.scatter(range(len(X)), [ii + 1.5] * len(X),
               c=y, marker='_', lw=lw, cmap=plt.cm.Set3)

    ax.scatter(range(len(X)), [ii + 2.5] * len(X),
               c=group, marker='_', lw=lw, cmap=cmap_data)
    
    if cv.stacking_mode:
        ax.scatter(range(len(X)), [ii + 3.5] * len(X),
               c=group, marker='_', lw=lw, cmap=cmap_data)

    # Formatting
    yticklabels = list(range(n_splits)) + ['target', 'day']
    ax.set(yticks=np.arange(n_splits+2) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+2.2, -.2], xlim=[0, len(y)])
    
    ax.set_title('{}'.format(name_dict[cv.stacking_mode]), fontsize=15)
    #ax.set_title('{}'.format(type(cv).__name__), fontsize=15)
    return ax

In [ ]:
del train,X,y

In [ ]:
#fig, ax = plt.subplots(1, 1, figsize = (20, 12))

#plot_cv_indices_stacking(cv, train[to_be_selected], train['action'], train['date'], 
#                         Sax, 5, lw=20)

### Optuna optimization + purged cv 

In [ ]:
N_SPLITS = 5
STACKING_MODE = True
VAL_GROUP_GAP = 20 #Days between end of training set and start of validation set
TEST_GROUP_GAP = 20 #Days between end of validation set and start of testing/stacking set
MAX_DAYS_TRAIN = 120
MAX_DAYS_VAL = 60
MAX_DAYS_TEST = 60
RANDOM_SEED = 28

In [ ]:
cv = PurgedGroupTimeSeriesSplitStacking(n_splits=N_SPLITS,
    stacking_mode=STACKING_MODE,
    max_train_group_size=MAX_DAYS_TRAIN, max_val_group_size=MAX_DAYS_VAL,
    max_test_group_size=MAX_DAYS_TEST, val_group_gap=VAL_GROUP_GAP,
    test_group_gap=TEST_GROUP_GAP)

# Use the following to test your pipeline
cv_dummy = PurgedGroupTimeSeriesSplitStacking(n_splits=2,
    stacking_mode=STACKING_MODE,
    max_train_group_size=1, max_val_group_size=1,
    max_test_group_size=1, val_group_gap=1,
    test_group_gap=1)

In [ ]:
#from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import AdaBoostClassifier
#from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression # 메타 모델
#from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
#import xgboost as xgb
#import lightgbm as lgb 
from sklearn.ensemble import RandomForestClassifier as rf


N_TRIALS=5
def objective(trial, cv=cv_dummy):
    
    #xgb parameters
    param_xgb = {
        "verbosity": 0,
        "tree_method": "gpu_hist",
        "objective": "binary:logistic",
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        "max_depth" : trial.suggest_int("max_depth", 1, 9),
        "learning_rate" : trial.suggest_float("eta", 1e-8, 1.0, log=True),
        "gamma" : trial.suggest_float("gamma", 1e-8, 1.0, log=True),
        "grow_policy" : trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    }


    param_lgb = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbose": -1,
        "boosting_type": "gbdt",
        "device" : "gpu",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    model_1 = XGBClassifier(**param_xgb)
    model_2 = LGBMClassifier(**param_lgb)
    final_estimator = RandomForestClassifier()
    
    # Ensemble bass models
    models = [
    ('xgb',model_1),
    ('lgb',model_2),
    #('rf',rfclf)
    ]

    #stack_clf = StackingClassifier(models,final_estimator=final_estimator)   
    stack_clf = VotingClassifier(estimators=models,voting='hard',weights=[0.35, 0.65],n_jobs=-1)

    val_aucs = []
    aucs = []
    for kfold, (train_idx, val_idx, test_idx) in enumerate(cv.split(X_train, 
                                                                    y_train, 
                                                                    groups)):

        %time stack_clf.fit(X_train, y_train)
    
        stack_final_pred = stack_clf.predict(X_test) 
        auc = roc_auc_score(y_test, stack_final_pred)
        print('Classifier: {}\tFold: {}\t AUC: {}\n'.format(type(final_estimator).__name__, kfold, auc))
        aucs.append(auc)
    
    print('Average AUC: {}'.format(np.average(auc)))
    return np.average(aucs)


In [ ]:
import optuna

study = optuna.create_study(study_name = 'stacking_parameter_opt', direction="maximize")
study.optimize(objective, n_trials=N_TRIALS) #Here use N_TRIALS when doing it properly

trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
# Classifier: AdaBoostClassifier	Fold: 1	 AUC: 0.5887123760591787

# Average AUC: 0.5887123760591787
#   Value: 0.5887322343881863
best_param = {
    'booster': 'dart',
    'lambda':0.02269898576617538,
    'alpha': 1.7519279647804202e-07,
    'max_depth': 1,
    'eta': 5.853592180789002e-07,
    'gamma': 0.00010007997910296745,
    'grow_policy': 'lossguide',
    'lambda_l1': 0.10062669044437277,
    'lambda_l2': 2.7130351663800012e-08,
    'num_leaves': 255,
    'feature_fraction':0.9163826994819885,
    'bagging_fraction':0.8581532897936976,
    'bagging_freq': 5,
    'min_child_samples': 70,
    'tree_method' : 'gpu_hist',
    'device': 'gpu',
    'subsample': 0.3
}

# best_param = {
#     'lambda': 2.4991265196872308e-08,
#     'alpha': 0.08940927931061919,
#     'max_depth': 1,
#     'eta': 1.4499980913381051e-06,
#     'gamma': 1.3137575007318098e-05,
#     'lambda_l1': 0.00010104042809601234,
#     'lambda_l2': 1.4677308343245044e-07,
#     'num_leaves': 235,
#     'feature_fraction': 0.6740934145011883,
#     'bagging_fraction': 0.7732904443272498,
#     'bagging_freq': 6,
#     'min_child_samples': 45,
#     'subsample' : 0.3 
# }



In [ ]:
#best_param = study.best_trial.params.items()
model_1 = XGBClassifier(**best_param)
model_2 = LGBMClassifier(**best_param)
final_estimator = RandomForestClassifier()

# Ensemble bass models
models = [
('xgb',model_1),
('lgb',model_2),
#('rf',rfclf)
]

stack_clf_best = VotingClassifier(estimators=models,voting='hard',weights=[0.35, 0.65],n_jobs=-1)
stack_clf_best.fit(X_train, y_train)

#del X_train, y_train

In [ ]:
save_pickle(stack_clf_best, './stacking_purgedcv4.npy')

In [ ]:
print(type(stack_clf_best))

In [ ]:
!pip install hummingbird-ml

In [ ]:
from hummingbird.ml import convert
stack_pytorch = convert(stack_clf_best, 'pytorch',None,'gpu')
print(type(stack_pytorch))

In [ ]:
save_pickle(stack_pytorch, './stacking_pytorch.npy')

In [ ]:
from tqdm import tqdm
import janestreet
env = janestreet.make_env() 
iter_test = env.iter_test() 

In [ ]:
th = 0.503
f = np.median

for (test_df, sample_prediction_df) in tqdm(iter_test):
    if test_df['weight'].item() > 0 :
        x_tt = test_df.loc[:, to_be_selected].values     
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt) * f_mean
        y_preds = stack_clf.predict(x_tt)
        sample_prediction_df.action = np.where(y_preds >= th, 1,0).astype(int)
    else:
        sample_prediction_df.action = 0
    env.predict(sample_prediction_df)

### Utility function

In [ ]:
X_utility = X_utility.loc[X_test.index,['date','resp','weight','action']]

date = X_utility['date'].values
weight = X_utility['weight'].values
resp = X_utility['resp'].values
action = X_utility['action'].values

#import njit
from numba import njit

@njit(fastmath = True)
def utility_score_numba(date, weight, resp, action):
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / len(Pi))
    u = min(max(t, 0), 6) * np.sum(Pi)
    return u
utility_score_numba(date, weight, resp, action)/44000 # * (public test set size)